# Scrapy
- https://scrapy.org/
- scrapy 기본 구조
- xpath
- Scrapy Project
- Scrapy Excute
- Pipelines

install scrapy
- `$ pip3 install scrapy`
- `conda install -c conda-forge scrapy `

### xpath
- scrapy shell을 이용해서도 가능
- `$ scrapy shell "https://www.naver.com/"`

In [1]:
import scrapy

In [2]:
import requests
from scrapy.http import TextResponse

In [3]:
# 웹 페이지에 연결
req = requests.get('https://www.naver.com/')

# response 객체 생성
response = TextResponse(req.url, body=req.text, encoding='utf-8')

xpath를 이용하여 특정 selector 객체 가져옴
- xpath : xpath 데이터
- data : xpath에서 선택된 위치의 데이터

- `//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[16]/a/span[2]`

- `//` : 가장 상위 Element
- `.` : 현재 Element
- `*` : 조건에 맞는 앞부분의 하위 Element를 모두 살펴봄 ( css selector에서 한칸 띄기와 같음 )
- `/` : 바로 아래 요소 ( css selector에서 > 와 같음 )
- `element[조건]`
    - p[2] : p element의 두번째 element : index가 0이 아닌 1부터 시작
    - [@(attribute key)="(attribute value)"]
    - [@id="email"] : 아이디 값이 email인 element
    - [@class="pw"] : 클래스 값이 pw인 element
- `not`
    - not(조건) : 조건이 아닌 요소를 찾음

In [4]:
# 네이버 키워드 순위 데이터 가져오기
response.xpath('//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span')

[<Selector xpath='//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span' data='<span class="ah_r">1</span>'>,
 <Selector xpath='//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span' data='<span class="ah_k">박대</span>'>]

In [5]:
# text
# xpath 가장 뒤에 text() 함수를 작성하면 text 데이터가 있는 object로 변경
response.xpath('//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span/text()')

[<Selector xpath='//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span/text()' data='1'>,
 <Selector xpath='//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span/text()' data='박대'>]

In [6]:
# extract
# xpath로 select된 element의 데이터를 문자열 리스트로 가져옴
response.xpath('//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li[1]/a/span/text()').extract()

['1', '박대']

##### 여러개 selecte 해서 데이터 가져오기
- 네이버 실시간 키워드

In [7]:
naver_keywords = response.xpath('//*[@id="PM_ID_ct"]/div[1]/div[2]/div[2]/div[1]/div/ul/li')
len(naver_keywords)

20

In [8]:
naver_rank = naver_keywords.xpath('./a/span[1]/text()').extract()[:10]
naver_rank

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [9]:
naver_keywords_list = naver_keywords.xpath('./a/span[2]/text()').extract()[:10]
naver_keywords_list

['박대', '식품안전나라', '도도맘', '강용석', '반건조박대', '박대구이', '김승현', '이호진', '임창용', '원할머니보쌈']

- 다음 실시간 키워드

In [10]:
# 웹 페이지에 연결
req = requests.get('https://daum.net/')

# response 객체 생성
response = TextResponse(req.url, body=req.text, encoding='utf-8')

In [11]:
# 순위
response.xpath('//*[@id="mArticle"]/div[2]/div[1]/div[2]/div[1]/ol/li/div/div[1]/span[1]/span/text()').extract()

['1위', '2위', '3위', '4위', '5위', '6위', '7위', '8위', '9위', '10위']

In [12]:
# 키워드
response.xpath('//*[@id="mArticle"]/div[2]/div[1]/div[2]/div[1]/ol/li/div/div[1]/span[2]/a/text()').extract()

['박대구이',
 '김승현',
 '강용석 구속',
 '도도맘 김미나',
 '태진아',
 '이창호',
 '손예진',
 '신현준',
 '김수미',
 '실화탐사대']

### Scrapy Project
- g마켓 베스트 셀러 상품, 상품평
- http://corners.gmarket.co.kr/Bestsellers

- crawler 프로젝트 실행
    - `$ scrapy startproject crawler`

In [13]:
!scrapy startproject crawler

New Scrapy project 'crawler', using template directory '/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /Users/rada/study/gitlab/dss/06_crawling/crawler

You can start your first spider with:
    cd crawler
    scrapy genspider example example.com


In [16]:
!tree crawler

crawler
├── crawler
│   ├── __init__.py
│   ├── __pycache__
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       └── __pycache__
└── scrapy.cfg

4 directories, 7 files


### scrapy 기본 구조

- Spider
    - 어떤 웹사이트들을 어떻게 크롤링할 것인지 명시하고, 각각의 웹 페이지의 어떤 부분을 스크래핑할 것인지 명시하는 클래스

- items.py
    - 웹페이지에서 원하는 부분을 스크랩하여 저장할 때 사용하는 사용자 정의 자료구조 클래스

- pipelines.py
    - 스크래핑 결과물을 Item 형태로 구성하였을 때, 이를 자유롭게 가공하거나 다양한 파일 형태로 저장할 수 있도록 하는 클래스 (getter와 setter의 개념)

- settings.py
    - Spider나 Item Pipeline 등이 어떻게 동작하도록 할 지에 대한 세부적인 설정 사항을 기재하는 파일 크롤링의 빈도등을 설정
    - cf. robots.txt (settings.py - ROBOTSTXT_OBEY = True)

- 자동으로 디렉토리와 파이썬 파일을 아래와 같이 만들어 준다.
```
.
├── crawler
│   ├── __init__.py
│   ├── __pycache__
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       └── __pycache__
└── scrapy.cfg
```

- 링크 크롤링 준비
- 크롤링 순서
    - 베스트 셀러 상품 200개에 대한 링크를 크롤링
    - 각 링크 내부 상세 페이지에서 제목, 판매가, 원가 정보를 가져옴
    - 할인률을 추가하여 스크래피로 크롤링

In [17]:
# 베스트 셀러 상품 200개의 엘리먼트를 selecting
req = requests.get('http://corners.gmarket.co.kr/Bestsellers')
response = TextResponse(req.url, body=req.text, encoding='utf-8')
bestsellers = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li')
len(bestsellers)

200

In [18]:
# 타이틀 데이터를 가져옴
titles = bestsellers.xpath('./a/text()').extract()
len(titles), titles[0]

(200, '(M)5.5리터 일렉트로맨 에어프라이어 (AFG-2801)')

In [19]:
# 상품 200개의 엘리먼트에서 링크 데이터를 가져옴
# links가 path로 되어 있을때, 전체 URL을 얻으려면 response.urljoin(path)로 실행
links = bestsellers.xpath('./a/@href').extract()
len(links), links[0]

(200, 'http://item.gmarket.co.kr/Item?goodscode=1496706894')

In [20]:
# 링크 데이터를 가져옴
bestsellers = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li')[:5]
for bestseller in bestsellers:
    link = bestseller.xpath('./a/@href').extract()
    print(link)

['http://item.gmarket.co.kr/Item?goodscode=1496706894']
['http://item.gmarket.co.kr/Item?goodscode=1507412334']
['http://item.gmarket.co.kr/Item?goodscode=336006717']
['http://item.gmarket.co.kr/Item?goodscode=1125887934']
['http://item.gmarket.co.kr/Item?goodscode=1246334162']


- 상세 페이지 크롤링

In [21]:
link

['http://item.gmarket.co.kr/Item?goodscode=1246334162']

In [24]:
# 상세 페이지 접속
req = requests.get(link[0])
response = TextResponse(req.url, body=req.text, encoding='utf-8')

In [25]:
# 제목
title = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract().strip()
title

'[르헤브] 1+1 지퍼형 구름 베개솜/피톤치드/다운필/메모리폼'

In [26]:
# 판매가
s_price = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract().replace(",","")
s_price

'5900'

In [27]:
# 원가
o_price = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract().replace(",","")
o_price

'29500'

In [28]:
# 할인율
discount_rate = str(round((1 - int(s_price) / int(o_price)) * 100, 2)) + "%"
discount_rate

'80.0%'

- crawler/items.py

```
import scrapy

class CrawlerItem(scrapy.Item):
    title = scrapy.Field()
    s_price = scrapy.Field()
    o_price = scrapy.Field()
    discount_rate = scrapy.Field()
```

In [32]:
%%writefile crawler/crawler/items.py
import scrapy

class CrawlerItem(scrapy.Item):
    title = scrapy.Field()
    s_price = scrapy.Field()
    o_price = scrapy.Field()
    discount_rate = scrapy.Field()

Overwriting crawler/crawler/items.py


In [33]:
!cat crawler/crawler/items.py

import scrapy

class CrawlerItem(scrapy.Item):
    title = scrapy.Field()
    s_price = scrapy.Field()
    o_price = scrapy.Field()
    discount_rate = scrapy.Field()

- crawler/spiders/spider.py

```
import scrapy

from crawler.items import CrawlerItem

# start_urls -> parse -> parse_page_contents 순으로 호출
class Spider(scrapy.Spider):
    # 스파이더의 이름
    name = "GmarketBestsellers"
    allow_domain = ["gmarket.co.kr"]
    # 크롤링의 시작 URL
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]

    # link 리스트를 가져옴
    def parse(self, response):
        bestsellers = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li')[:10]
        for bestseller in bestsellers:
            link = bestseller.xpath('./a/@href').extract()[0]
            # 각 링크마다 parse_page_contents 함수를 호출
            yield scrapy.Request(link, callback=self.parse_page_contents)

    # 각페이지의 link로 접속하여 데이터를 가져옴
    def parse_page_contents(self, response):
        item = CrawlerItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract().strip()
        item["s_price"] = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract().replace(",","")
        # 원가가 없는 경우 판매가를 넣음
        try:
            item["o_price"] = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract().replace(",","")
        except:
            item["o_price"] = item["s_price"]
        item["discount_rate"] = str(round((1 - int(item["s_price"]) / int(item["o_price"])) * 100, 2)) + "%"
        yield item
```

In [34]:
%%writefile crawler/crawler/spiders/spider.py
import scrapy

from crawler.items import CrawlerItem

# start_urls -> parse -> parse_page_contents 순으로 호출
class Spider(scrapy.Spider):
    # 스파이더의 이름
    name = "GmarketBestsellers"
    allow_domain = ["gmarket.co.kr"]
    # 크롤링의 시작 URL
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]

    # link 리스트를 가져옴
    def parse(self, response):
        bestsellers = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li')[:10]
        for bestseller in bestsellers:
            link = bestseller.xpath('./a/@href').extract()[0]
            # 각 링크마다 parse_page_contents 함수를 호출
            yield scrapy.Request(link, callback=self.parse_page_contents)

    # 각페이지의 link로 접속하여 데이터를 가져옴
    def parse_page_contents(self, response):
        item = CrawlerItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract().strip()
        item["s_price"] = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract().replace(",","")
        # 원가가 없는 경우 판매가를 넣음
        try:
            item["o_price"] = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract().replace(",","")
        except:
            item["o_price"] = item["s_price"]
        item["discount_rate"] = str(round((1 - int(item["s_price"]) / int(item["o_price"])) * 100, 2)) + "%"
        yield item

Writing crawler/crawler/spiders/spider.py


In [35]:
!cat crawler/crawler/spiders/spider.py

import scrapy

from crawler.items import CrawlerItem

# start_urls -> parse -> parse_page_contents 순으로 호출
class Spider(scrapy.Spider):
    # 스파이더의 이름
    name = "GmarketBestsellers"
    allow_domain = ["gmarket.co.kr"]
    # 크롤링의 시작 URL
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]

    # link 리스트를 가져옴
    def parse(self, response):
        bestsellers = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li')[:10]
        for bestseller in bestsellers:
            link = bestseller.xpath('./a/@href').extract()[0]
            # 각 링크마다 parse_page_contents 함수를 호출
            yield scrapy.Request(link, callback=self.parse_page_contents)

    # 각페이지의 link로 접속하여 데이터를 가져옴
    def parse_page_contents(self, response):
        item = CrawlerItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract().strip()
        item["s_price"] = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract().replace(",","")
        # 

### Scrapy Excute
- scrapy.cfg 파일이 있는 디렉토리에서 아래의 명령을 실행하면 크롤링이 시작됨
    - `$ scrapy crawl GmarketBestsellers`
- 결과를 csv 파일로 저장
    - `$ scrapy crawl GmarketBestsellers -o GmarketBestsellers.csv`
    - 한꺼번에 페이지를 로드해서 가져옴으로 저장되는 순서가 일정하지 않음
    - 컬럼 데이터도 순서가 뒤죽박죽

In [46]:
!ls crawler/

crawler    scrapy.cfg


In [49]:
%%writefile run.sh
cd crawler
scrapy crawl GmarketBestsellers

Overwriting run.sh


In [50]:
!source run.sh

2018-10-24 21:58:32 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: crawler)
2018-10-24 21:58:32 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 3.6.4 (default, Jan  6 2018, 11:51:59) - [GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.39.2)], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Darwin-17.7.0-x86_64-i386-64bit
2018-10-24 21:58:32 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'crawler', 'NEWSPIDER_MODULE': 'crawler.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['crawler.spiders']}
2018-10-24 21:58:32 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2018-10-24 21:58:32 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddle

In [52]:
%%writefile run.sh
cd crawler
scrapy crawl GmarketBestsellers -o GmarketBestsellers.csv

Overwriting run.sh


In [53]:
!source run.sh

2018-10-24 22:00:56 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: crawler)
2018-10-24 22:00:56 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 3.6.4 (default, Jan  6 2018, 11:51:59) - [GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.39.2)], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Darwin-17.7.0-x86_64-i386-64bit
2018-10-24 22:00:56 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'crawler', 'FEED_FORMAT': 'csv', 'FEED_URI': 'GmarketBestsellers.csv', 'NEWSPIDER_MODULE': 'crawler.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['crawler.spiders']}
2018-10-24 22:00:56 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2018-10-24 22:00:56 [scrapy.

In [54]:
!ls crawler/

GmarketBestsellers.csv crawler                scrapy.cfg


In [57]:
df = pd.read_csv("crawler/GmarketBestsellers.csv")
df

discount_rate  o_price  s_price                               title
0        60.24%    24900     9900       손님 초대요리로 좋은 밀푀유나베(2인) KIT 밀키트
1        55.28%    19900     8900               국내산 활가리비  2kg(40-60미)
2          0.0%    33900    33900     하루활력 여성건강 시크릿 석류 70ml 30팩 1+1증정
3         6.11%    89800    84310    (M)5.5리터 일렉트로맨 에어프라이어 (AFG-2801)
4         70.0%    23000     6900         국산/가을신상/아동/내복/내의/주니어/실내복/잠옷
5         80.0%    29500     5900  [르헤브] 1+1 지퍼형 구름 베개솜/피톤치드/다운필/메모리폼
6         15.5%    12900    10900       여성 양털 후드 짚업 남녀공용 핫뜨거 후리스 양털후드
7         45.5%    20000    10900         붙이는 핫팩 100매 파스형핫팩/안전인증/허리핫팩
8        55.14%   109000    48900        [팸퍼스] 팸퍼스 베이비드라이 팬티4팩 기저귀(슈)
9        38.12%    16000     9900        나영이네농장 맛좋은 감귤 10kg(L~2L)-중대과

### Pipelines
- 결과를 깔끔하게 정리해서 저장
- spider의 items 데이터가 pipelines에 정의된 대로 저장

- pipelines.py 

```
import csv

class CrawlerPipeline(object):

    def __init__(self):
        self.csvwriter = csv.writer(open("GmarketBestsellers.csv", "w"))
        self.csvwriter.writerow(["title","o_price","s_price","discount_rate"])

    def process_item(self, item, spider):
        row = []
        row.append(item["title"])
        row.append(item["o_price"])
        row.append(item["s_price"])
        row.append(item["discount_rate"])
        self.csvwriter.writerow(row)
        return item
```

In [58]:
%%writefile crawler/crawler/pipelines.py
import csv

class CrawlerPipeline(object):

    def __init__(self):
        self.csvwriter = csv.writer(open("GmarketBestsellers.csv", "w"))
        self.csvwriter.writerow(["title","o_price","s_price","discount_rate"])

    def process_item(self, item, spider):
        row = []
        row.append(item["title"])
        row.append(item["o_price"])
        row.append(item["s_price"])
        row.append(item["discount_rate"])
        self.csvwriter.writerow(row)
        return item

Overwriting crawler/crawler/pipelines.py


In [59]:
!cat crawler/crawler/pipelines.py

import csv

class CrawlerPipeline(object):

    def __init__(self):
        self.csvwriter = csv.writer(open("GmarketBestsellers.csv", "w"))
        self.csvwriter.writerow(["title","o_price","s_price","discount_rate"])

    def process_item(self, item, spider):
        row = []
        row.append(item["title"])
        row.append(item["o_price"])
        row.append(item["s_price"])
        row.append(item["discount_rate"])
        self.csvwriter.writerow(row)
        return item

- settings.py 에 아래 코드 추가

```
ITEM_PIPELINES = {
   'crawler.pipelines.CrawlerPipeline': 300,
}
```

In [68]:
!echo "ITEM_PIPELINES = {" >> crawler/crawler/settings.py

In [69]:
!echo "   'crawler.pipelines.CrawlerPipeline': 300," >> crawler/crawler/settings.py

In [70]:
!echo "}" >> crawler/crawler/settings.py

In [71]:
!tail -n 3 crawler/crawler/settings.py

ITEM_PIPELINES = {
   'crawler.pipelines.CrawlerPipeline': 300,
}


In [72]:
%%writefile run.sh
cd crawler
scrapy crawl GmarketBestsellers

Overwriting run.sh


In [73]:
!source run.sh

2018-10-24 22:10:50 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: crawler)
2018-10-24 22:10:50 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 3.6.4 (default, Jan  6 2018, 11:51:59) - [GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.39.2)], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Darwin-17.7.0-x86_64-i386-64bit
2018-10-24 22:10:50 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'crawler', 'NEWSPIDER_MODULE': 'crawler.spiders', 'ROBOTSTXT_OBEY': True, 'SPIDER_MODULES': ['crawler.spiders']}
2018-10-24 22:10:50 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2018-10-24 22:10:50 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddle

In [74]:
df = pd.read_csv("crawler/GmarketBestsellers.csv")
df

title  o_price  s_price discount_rate
0        나영이네농장 맛좋은 감귤 10kg(L~2L)-중대과    16000     9900        38.12%
1     반건조 군산 박대 서대 6~10미 600~700g 국내산    21900    21900          0.0%
2  [르헤브] 1+1 지퍼형 구름 베개솜/피톤치드/다운필/메모리폼    29500     5900         80.0%
3         붙이는 핫팩 100매 파스형핫팩/안전인증/허리핫팩    20000    10900         45.5%
4               국내산 활가리비  2kg(40-60미)    19900     8900        55.28%
5         국산/가을신상/아동/내복/내의/주니어/실내복/잠옷    23000     6900         70.0%
6     하루활력 여성건강 시크릿 석류 70ml 30팩 1+1증정    33900    33900          0.0%
7       여성 양털 후드 짚업 남녀공용 핫뜨거 후리스 양털후드    12900    10900         15.5%
8        [팸퍼스] 팸퍼스 베이비드라이 팬티4팩 기저귀(슈)   109000    48900        55.14%
9       손님 초대요리로 좋은 밀푀유나베(2인) KIT 밀키트    24900     9900        60.24%

### 네이버 현재 상영 영화 제목, 관객수, 별점 크롤링

- 프로젝트 생성

```
$ scrapy startproject naver_movie
```

In [75]:
!scrapy startproject naver_movie

New Scrapy project 'naver_movie', using template directory '/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /Users/rada/study/gitlab/dss/06_crawling/naver_movie

You can start your first spider with:
    cd naver_movie
    scrapy genspider example example.com


In [76]:
!tree naver_movie/

naver_movie/
├── naver_movie
│   ├── __init__.py
│   ├── __pycache__
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       └── __pycache__
└── scrapy.cfg

4 directories, 7 files


- items.py

```
import scrapy


class NaverMovieItem(scrapy.Item):
    title = scrapy.Field()
    count = scrapy.Field()
    star = scrapy.Field()
```

In [1]:
%%writefile naver_movie/naver_movie/items.py
import scrapy


class NaverMovieItem(scrapy.Item):
    title = scrapy.Field()
    count = scrapy.Field()
    star = scrapy.Field()

Overwriting naver_movie/naver_movie/items.py


In [2]:
!cat naver_movie/naver_movie/items.py

import scrapy


class NaverMovieItem(scrapy.Item):
    title = scrapy.Field()
    count = scrapy.Field()
    star = scrapy.Field()

- 가져올 데이터 확인

In [84]:
# 링크
req = requests.get('https://movie.naver.com/movie/running/current.nhn')
response = TextResponse(req.url, body=req.text, encoding='utf-8')
links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href').extract()
links[:5]

['/movie/bi/mi/basic.nhn?code=167105',
 '/movie/bi/mi/basic.nhn?code=168058',
 '/movie/bi/mi/basic.nhn?code=155356',
 '/movie/bi/mi/basic.nhn?code=119428',
 '/movie/bi/mi/basic.nhn?code=168050']

In [85]:
# 도메인 추가
link = response.urljoin(links[0])
link

'https://movie.naver.com/movie/bi/mi/basic.nhn?code=167105'

In [86]:
# 제목과 관객수
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding='utf-8')
title = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
count = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0]
star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
star = "".join(star)

title, count, star

('암수살인', '3,472,278명', '8.60')

- spiders/spider.py

```
import scrapy
from naver_movie.items import NaverMovieItem

class MovieSpider(scrapy.Spider):
    name = "NaverMovie"
    allow_domain = ["https://movie.naver.com"]
    start_urls = ["https://movie.naver.com/movie/running/current.nhn"]

    def parse(self, response):
        links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href')[:10].extract()
        for link in links:
            link = response.urljoin(link)
            yield scrapy.Request(link, callback=self.parse_page_contents)

    def parse_page_contents(self, response):
        item = NaverMovieItem()
        item["title"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
        try:
            item["count"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0]
        except:
            item["count"] = "0명"
        star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
        item["star"] = "".join(star)
        yield item
```

In [3]:
%%writefile naver_movie/naver_movie/spiders/spider.py
import scrapy
from naver_movie.items import NaverMovieItem

class MovieSpider(scrapy.Spider):
    name = "NaverMovie"
    allow_domain = ["https://movie.naver.com"]
    start_urls = ["https://movie.naver.com/movie/running/current.nhn"]

    def parse(self, response):
        links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href')[:10].extract()
        for link in links:
            link = response.urljoin(link)
            yield scrapy.Request(link, callback=self.parse_page_contents)

    def parse_page_contents(self, response):
        item = NaverMovieItem()
        item["title"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
        try:
            item["count"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0]
        except:
            item["count"] = "0명"
        star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
        item["star"] = "".join(star)
        yield item

Overwriting naver_movie/naver_movie/spiders/spider.py


In [4]:
!cat naver_movie/naver_movie/spiders/spider.py

import scrapy
from naver_movie.items import NaverMovieItem

class MovieSpider(scrapy.Spider):
    name = "NaverMovie"
    allow_domain = ["https://movie.naver.com"]
    start_urls = ["https://movie.naver.com/movie/running/current.nhn"]

    def parse(self, response):
        links = response.xpath('//*[@id="content"]/div[1]/div[1]/div[3]/ul/li/dl/dt/a/@href')[:10].extract()
        for link in links:
            link = response.urljoin(link)
            yield scrapy.Request(link, callback=self.parse_page_contents)

    def parse_page_contents(self, response):
        item = NaverMovieItem()
        item["title"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
        try:
            item["count"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[5]/div/p[2]/text()').extract()[0]
        except:
            item["count"] = "0명"
        star = response.xpath('//*[@id="actualPointPersentBasic"]/div/em/text()').extract()
        item["

- 크롤링 실행

```
$ scrapy crawl NaverMovie
$ scrapy crawl NaverMovie -o naver_movie.csv
```

In [5]:
%%writefile run.sh
cd naver_movie
scrapy crawl NaverMovie

Overwriting run.sh


In [6]:
!source run.sh

2018-10-26 12:55:47 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: naver_movie)
2018-10-26 12:55:47 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 3.6.4 (default, Jan  6 2018, 11:51:59) - [GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.39.2)], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Darwin-17.7.0-x86_64-i386-64bit
2018-10-26 12:55:47 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'naver_movie', 'NEWSPIDER_MODULE': 'naver_movie.spiders', 'SPIDER_MODULES': ['naver_movie.spiders']}
2018-10-26 12:55:47 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2018-10-26 12:55:47 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 's

- settings.py

```
ROBOTSTXT_OBEY = False
```

In [94]:
!cat naver_movie/naver_movie/settings.py

# -*- coding: utf-8 -*-

# Scrapy settings for naver_movie project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://doc.scrapy.org/en/latest/topics/settings.html
#     https://doc.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://doc.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'naver_movie'

SPIDER_MODULES = ['naver_movie.spiders']
NEWSPIDER_MODULE = 'naver_movie.spiders'


# Crawl responsibly by identifying yourself (and your website) on the user-agent
#USER_AGENT = 'naver_movie (+http://www.yourdomain.com)'

# Obey robots.txt rules
ROBOTSTXT_OBEY = True

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32

# Configure a delay for requests for the same website (default: 0)
# See https://doc.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and d

In [106]:
!sed -i .bak 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' naver_movie/naver_movie/settings.py

In [107]:
!ls naver_movie/naver_movie/

__init__.py     items.py        pipelines.py    settings.py.bak
__pycache__     middlewares.py  settings.py     spiders


In [108]:
!cat naver_movie/naver_movie/settings.py

# -*- coding: utf-8 -*-

# Scrapy settings for naver_movie project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://doc.scrapy.org/en/latest/topics/settings.html
#     https://doc.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://doc.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'naver_movie'

SPIDER_MODULES = ['naver_movie.spiders']
NEWSPIDER_MODULE = 'naver_movie.spiders'


# Crawl responsibly by identifying yourself (and your website) on the user-agent
#USER_AGENT = 'naver_movie (+http://www.yourdomain.com)'

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32

# Configure a delay for requests for the same website (default: 0)
# See https://doc.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and 

다시 실행

In [109]:
!source run.sh

2018-10-24 22:39:23 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: naver_movie)
2018-10-24 22:39:23 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 3.6.4 (default, Jan  6 2018, 11:51:59) - [GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.39.2)], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Darwin-17.7.0-x86_64-i386-64bit
2018-10-24 22:39:23 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'naver_movie', 'NEWSPIDER_MODULE': 'naver_movie.spiders', 'SPIDER_MODULES': ['naver_movie.spiders']}
2018-10-24 22:39:23 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2018-10-24 22:39:23 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 's

In [110]:
%%writefile run.sh
cd naver_movie
scrapy crawl NaverMovie -o naver_movie.csv

Overwriting run.sh


In [7]:
!source run.sh

2018-10-26 12:56:12 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: naver_movie)
2018-10-26 12:56:12 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 3.6.4 (default, Jan  6 2018, 11:51:59) - [GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.39.2)], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Darwin-17.7.0-x86_64-i386-64bit
2018-10-26 12:56:12 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'naver_movie', 'NEWSPIDER_MODULE': 'naver_movie.spiders', 'SPIDER_MODULES': ['naver_movie.spiders']}
2018-10-26 12:56:12 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2018-10-26 12:56:12 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 's

In [112]:
df = pd.read_csv('naver_movie/naver_movie.csv')
df

count  star            title
0  3,472,278명  8.60             암수살인
1    466,502명  8.96  곰돌이 푸 다시 만나 행복해
2    288,324명  9.17          스타 이즈 본
3    498,220명  9.24              미쓰백
4          0명  8.64              안시성
5     28,247명  7.35          펭귄 하이웨이
6  3,718,592명  8.27               베놈
7          0명  8.32               액슬
8      9,086명  9.12         호밀밭의 반항아
9    486,194명  8.29             퍼스트맨